# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 8
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr8.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr8.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 222 222

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000172748,ENSG00000182372,ENSG00000104728,ENSG00000036448,ENSG00000091879,ENSG00000155189,ENSG00000164821,ENSG00000253958,ENSG00000147324,ENSG00000173281
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000172748,1.00000000,0.02931887,0.03472605,-0.02439109,0.00000000,0.00000000,0.00000000,0.0000000,0.00000000,0.00000000
ENSG00000182372,0.02931887,1.00000000,-0.06838250,0.08534388,-0.03524857,-0.05286252,0.00000000,0.0000000,0.00000000,0.00000000
ENSG00000104728,0.03472605,-0.06838250,1.00000000,0.02055591,-0.04310498,0.07790795,-0.02011401,0.0000000,0.00000000,0.00000000
ENSG00000036448,-0.02439109,0.08534388,0.02055591,1.00000000,-0.01193855,0.03715707,-0.05811610,0.0000000,0.00000000,0.00000000
ENSG00000091879,0.00000000,-0.03524857,-0.04310498,-0.01193855,1.00000000,0.02863605,0.02006410,0.0474210,-0.04648296,0.02190241
ENSG00000155189,0.00000000,-0.05286252,0.07790795,0.03715707,0.02863605,1.00000000,-0.10798990,0.1888645,0.02620260,-0.05672479


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 8

[1] -0.01805292 -0.07109667 -0.08025913 -0.08215712 -0.26141577 -0.49281689
[7] -1.03522088 -1.26669554

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 222 222

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000172748,ENSG00000182372,ENSG00000104728,ENSG00000036448,ENSG00000091879,ENSG00000155189,ENSG00000164821,ENSG00000253958,ENSG00000147324,ENSG00000173281
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000172748,1.00000000,0.02931887,0.03472605,-0.02439109,0.00000000,0.00000000,0.00000000,0.0000000,0.00000000,0.00000000
ENSG00000182372,0.02931887,1.00000000,-0.06838250,0.08534388,-0.03524857,-0.05286252,0.00000000,0.0000000,0.00000000,0.00000000
ENSG00000104728,0.03472605,-0.06838250,1.00000000,0.02055591,-0.04310498,0.07790795,-0.02011401,0.0000000,0.00000000,0.00000000
ENSG00000036448,-0.02439109,0.08534388,0.02055591,1.00000000,-0.01193855,0.03715707,-0.05811610,0.0000000,0.00000000,0.00000000
ENSG00000091879,0.00000000,-0.03524857,-0.04310498,-0.01193855,1.00000000,0.02863605,0.02006410,0.0474210,-0.04648296,0.02190241
ENSG00000155189,0.00000000,-0.05286252,0.07790795,0.03715707,0.02863605,1.00000000,-0.10798990,0.1888645,0.02620260,-0.05672479


,ENSG00000172748,ENSG00000182372,ENSG00000104728,ENSG00000036448,ENSG00000091879,ENSG00000155189,ENSG00000164821,ENSG00000253958,ENSG00000147324,ENSG00000173281
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000172748,1.0000000000,0.001045143,0.0126676510,-0.0014158703,-0.007270520,-0.0006070921,0.003244942,0.0003554503,-0.0002523168,0.0015093936
ENSG00000182372,0.0010451426,1.000000000,-0.0312172085,0.0349986803,-0.015480467,-0.0304531729,0.004005112,-0.0073498478,0.0026469945,-0.0004888716
ENSG00000104728,0.0126676510,-0.031217209,1.0000000000,-0.0001901169,-0.007031286,0.0488929571,-0.028765897,0.0041621024,0.0082216977,-0.0064872737
ENSG00000036448,-0.0014158703,0.034998680,-0.0001901169,1.0000000000,-0.008365653,0.0241311924,-0.036694371,0.0027857639,0.0068484301,-0.0026449886
ENSG00000091879,-0.0072705202,-0.015480467,-0.0070312863,-0.0083656532,1.000000000,0.0067920325,0.011905261,0.0398552176,-0.0315584943,0.0271838781
ENSG00000155189,-0.0006070921,-0.030453173,0.0488929571,0.0241311924,0.006792032,1.0000000000,-0.050936064,0.0598471042,0.0321340519,-0.0391586288


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)